In [1]:
import psycopg2
import requests
import json
import pandas as pd
%run 'D:\\dev\\prod\\metrikaHelper.py'
def getVipProjectByDomain(domain):
    conn = psycopg2.connect("dbname='CRM' user='read_only' host='192.168.10.10' password='User_ro'")
    cur = conn.cursor()
    cur.execute("SELECT p.projectid, p.domain, p.status, p2.login, p2.password FROM project AS p INNER JOIN ProjectMetrikaData AS p2 ON p.UniqueId = p2.ProjectUniqueId WHERE domain=%s AND p.status<230 ORDER BY p.projectid", [domain])
    result = []
    rows = cur.fetchall()
    for row in rows:
        result.append(getVipProjectByRow(row))
    return result
def getVipProjectByRow(row):
    return {'projectid': row[0], 'domain': row[1], 'metrikalogin': row[3], 'metrikapassword': row[4]}
def getCPATargetsByProjectID(projectID):
    conn = psycopg2.connect("dbname='Cubo' user='read_only' host='192.168.10.8' password='User_ro'")
    cur = conn.cursor()
    cur.execute("SELECT targetstatus, targetid, targetname FROM metricstarget WHERE projectid=%s ORDER BY id DESC", [projectID])
    result = []
    rows = cur.fetchall()
    for row in rows:
        result.append(getCPATargetByRow(row))
    return result
def getCPATargetByRow(row):
    return {'targetstatus': row[0], 'targetid': row[1], 'targetname': row[2]}
def getGoalsByCounter(counterId,token):
    return requests.get('https://api-metrika.yandex.ru/management/v1/counter/{0}/goals?oauth_token={1}'.format(counterId,token)).text.encode('utf-8')
def getMetrikaReportInDataFrame(date1,date2,metrics,dimensions,token,counterId,filters,attribution):
    params = '&'.join(['dimensions={0}'.format(dimensions),
                       'metrics={0}'.format(metrics),
                       'date1={0}'.format(date1),
                      'date2={0}'.format(date2),
                      'ids={0}'.format(counterId),
                      'oauth_token={0}'.format(token),
                      'limit=100000',
                      'offset=1',
                      "filters={0}".format(filters),
                      'accuracy=full',
                      'attribution={0}'.format(attribution)])
    url = 'https://beta.api-metrika.yandex.ru/stat/v1/data.csv?'+params
    print url
    df = pd.read_csv(url,encoding='utf-8')
    return df

In [51]:
def mergeCPAVisitsColumns(df):
    for i in xrange(0,len(df)):
        df.loc[i,'goal_visits'] = df.loc[i,[column for column in df.columns if u'Целевые визиты' in column]].sum()
    df.drop([column for column in df.columns if u'Целевые визиты' in column], axis=1, inplace=True)
    return df
def dropTotalRow(df):
    return df.drop(df.index[0]).reset_index(drop=True)

In [4]:
domains = ["russj.ru","organickids.ru","pinakotekaresto.com","prosto-gruz.ru","zbbb.ru","alcotek.ru"]
all_df = pd.DataFrame()
for domain in domains:
    vipProjectInfo = getVipProjectByDomain(domain)
    metrikaCredentials = getMetrikaCredentialsByLPD({'yandexLogin':vipProjectInfo[0]['metrikalogin'],'yandexPassword':vipProjectInfo[0]['metrikapassword'],'domain':domain.decode("utf-8")})
    date1 = '2016-04-01'
    date2 = '2016-06-01'
    goalsMetrics = ','.join(['ym:s:goal'+str(target['targetid'])+'visits' for target in getCPATargetsByProjectID(vipProjectInfo[0]['projectid'])])
    otherMetrics = 'ym:s:visits'
    metrics = ','.join([otherMetrics,goalsMetrics])
    dimensions='ym:s:specialUser,ym:s:dateTime,ym:s:<attribution>TrafficSource,ym:s:<attribution>SourceEngine,ym:s:<attribution>UTMSource,ym:s:<attribution>UTMMedium,ym:s:<attribution>UTMCampaign,ym:s:<attribution>UTMContent,ym:s:<attribution>UTMTerm'
    token = metrikaCredentials['Token']
    counterId = metrikaCredentials['counterID'][0]
    attribution = 'lastsign'
    filters = ""
    original_data = getMetrikaReportInDataFrame(date1,date2,metrics,dimensions,token,counterId,filters,attribution)
    filtred_data = mergeCPAVisitsColumns(dropTotalRow(original_data))
    filtred_data.rename(columns={u'﻿""':'UserId'},inplace=True)
    usersdata = filtred_data[['UserId','goal_visits']].groupby(['UserId'],as_index=False).sum()
    new_data = filtred_data[filtred_data['UserId'].isin(usersdata[usersdata['goal_visits'] > 0]['UserId'])]
    userswithvisits = new_data['UserId'].value_counts()>1
    new_data = new_data[new_data['UserId'].isin(userswithvisits[userswithvisits == True].index)]
    userswithsources = new_data[['UserId',u'Источник трафика',u'Источник трафика (детально)']].drop_duplicates()['UserId'].value_counts() > 1
    new_data = new_data[new_data['UserId'].isin(userswithsources[userswithsources == True].index)]
    new_data['domain'] = domain
    all_df = pd.concat([all_df,new_data])
all_df.to_excel(u'D:\\_Docs_\\2016-06\\2016-06-06 Multichannel Funnels\\MCF_all_data.xlsx')

https://beta.api-metrika.yandex.ru/stat/v1/data.csv?dimensions=ym:s:specialUser,ym:s:dateTime,ym:s:<attribution>TrafficSource,ym:s:<attribution>SourceEngine,ym:s:<attribution>UTMSource,ym:s:<attribution>UTMMedium,ym:s:<attribution>UTMCampaign,ym:s:<attribution>UTMContent,ym:s:<attribution>UTMTerm&metrics=ym:s:visits,ym:s:goal20722165visits,ym:s:goal11692031visits,ym:s:goal11692036visits&date1=2016-04-01&date2=2016-06-01&ids=30722483&oauth_token=2cc70776f1ca4b06a662ec47714291e4&limit=100000&offset=1&filters=&accuracy=full&attribution=lastsign
https://beta.api-metrika.yandex.ru/stat/v1/data.csv?dimensions=ym:s:specialUser,ym:s:dateTime,ym:s:<attribution>TrafficSource,ym:s:<attribution>SourceEngine,ym:s:<attribution>UTMSource,ym:s:<attribution>UTMMedium,ym:s:<attribution>UTMCampaign,ym:s:<attribution>UTMContent,ym:s:<attribution>UTMTerm&metrics=ym:s:visits,ym:s:goal19493860visits,ym:s:goal11284707visits,ym:s:goal11284702visits,ym:s:goal11086937visits&date1=2016-04-01&date2=2016-06-01&ids=

In [103]:
filtred_data['goal_visits'].sum()

244.0

Тестирование атрибуции VVV

In [69]:
domains = ["russj.ru","organickids.ru","pinakotekaresto.com","prosto-gruz.ru","zbbb.ru","alcotek.ru","edemdom.ru","labotini.ru","realstroy.com","tencon.ru","жк-московский.com",
"народная-23.рф","doorlock.ru","stm66.ru","academoffice.ru","adel-omsk.ru","baxburo.ru","fabslog.ru","ferkon.ru","fosterland.ru","gemainvest.ru","hotel.sipk.ru",
"kerasol.ru","medstrahovka.ru","m-y.ru","nasoscentr.ru","optikdeal.ru","pickup-center.ru","proudmom.ru","ribambelle.ru","rusmaneken.ru","simple-pro.com","teploppu.ru",
"altpremium.ru","mosvipremont.ru","multifin.ru","megakran.ru","energotehmontag.ru","nanosegway.ru","mebmetall.ru","oooptr77.ru","saabparts.su","lodki-piter.ru",
"accessoriesforhome.ru","darion-svet.ru","vtb24leasing.ru","sevastopolstroy.ru"]

In [72]:
total_df = pd.DataFrame()
for domain in domains:
    vipProjectInfo = getVipProjectByDomain(domain)
    metrikaCredentials = getMetrikaCredentialsByLPD({'yandexLogin':vipProjectInfo[0]['metrikalogin'],'yandexPassword':vipProjectInfo[0]['metrikapassword'],'domain':domain.decode("utf-8")})

    date1 = '2016-05-01'
    date2 = '2016-05-31'
    goalsMetrics = ','.join(['ym:s:goal'+str(target['targetid'])+'visits' for target in getCPATargetsByProjectID(vipProjectInfo[0]['projectid'])])
    otherMetrics = 'ym:s:visits'
    metrics = ','.join([otherMetrics,goalsMetrics])
    dimensions=''
    token = metrikaCredentials['Token']
    counterId = metrikaCredentials['counterID'][0]
    filters = "ym:s:<attribution>UTMSource=='cubo'"
    attribution = 'last'
    df = getMetrikaReportInDataFrame(date1,date2,metrics,dimensions,token,counterId,filters,attribution)
    df['attribution'] = attribution
    df['domain'] = domain
    df = mergeCPAVisitsColumns(df)
    total_df = pd.concat([total_df,df])
    
    date1 = '2016-04-01'
    date2 = '2016-06-01'
    goalsMetrics = ','.join(['ym:s:goal'+str(target['targetid'])+'visits' for target in getCPATargetsByProjectID(vipProjectInfo[0]['projectid'])])
    otherMetrics = 'ym:s:visits'
    metrics = ','.join([otherMetrics,goalsMetrics])
    dimensions=''
    token = metrikaCredentials['Token']
    counterId = metrikaCredentials['counterID'][0]
    filters = "ym:s:<attribution>UTMSource=='cubo'"
    attribution = 'first'
    df = getMetrikaReportInDataFrame(date1,date2,metrics,dimensions,token,counterId,filters,attribution)
    df['attribution'] = attribution
    df['domain'] = domain
    df = mergeCPAVisitsColumns(df)
    total_df = pd.concat([total_df,df])
    
    date1 = '2016-04-01'
    date2 = '2016-06-01'
    goalsMetrics = ','.join(['ym:s:goal'+str(target['targetid'])+'visits' for target in getCPATargetsByProjectID(vipProjectInfo[0]['projectid'])])
    otherMetrics = 'ym:s:visits'
    metrics = ','.join([otherMetrics,goalsMetrics])
    dimensions=''
    token = metrikaCredentials['Token']
    counterId = metrikaCredentials['counterID'][0]
    filters = "ym:s:<attribution>UTMSource=='cubo'"
    attribution = 'lastsign'
    df = getMetrikaReportInDataFrame(date1,date2,metrics,dimensions,token,counterId,filters,attribution)
    df['attribution'] = attribution
    df['domain'] = domain
    df = mergeCPAVisitsColumns(df)
    total_df = pd.concat([total_df,df])
    
    date1 = '2016-04-01'
    date2 = '2016-06-01'
    goalsMetrics = ','.join(['ym:s:goal'+str(target['targetid'])+'visits' for target in getCPATargetsByProjectID(vipProjectInfo[0]['projectid'])])
    otherMetrics = 'ym:s:visits'
    metrics = ','.join([otherMetrics,goalsMetrics])
    dimensions=''
    token = metrikaCredentials['Token']
    counterId = metrikaCredentials['counterID'][0]
    filters = "ym:s:UTMSource=='cubo'"
    attribution = ''
    df = getMetrikaReportInDataFrame(date1,date2,metrics,dimensions,token,counterId,filters,attribution)
    df['attribution'] = 'default'
    df['domain'] = domain
    df = mergeCPAVisitsColumns(df)
    total_df = pd.concat([total_df,df])

https://beta.api-metrika.yandex.ru/stat/v1/data.csv?dimensions=&metrics=ym:s:visits,ym:s:goal11692031visits,ym:s:goal11692036visits&date1=2016-05-01&date2=2016-05-31&ids=30722483&oauth_token=2cc70776f1ca4b06a662ec47714291e4&limit=100000&offset=1&filters=ym:s:<attribution>UTMSource=='cubo'&accuracy=full&attribution=last
https://beta.api-metrika.yandex.ru/stat/v1/data.csv?dimensions=&metrics=ym:s:visits,ym:s:goal11692031visits,ym:s:goal11692036visits&date1=2016-04-01&date2=2016-06-01&ids=30722483&oauth_token=2cc70776f1ca4b06a662ec47714291e4&limit=100000&offset=1&filters=ym:s:<attribution>UTMSource=='cubo'&accuracy=full&attribution=first
https://beta.api-metrika.yandex.ru/stat/v1/data.csv?dimensions=&metrics=ym:s:visits,ym:s:goal11692031visits,ym:s:goal11692036visits&date1=2016-04-01&date2=2016-06-01&ids=30722483&oauth_token=2cc70776f1ca4b06a662ec47714291e4&limit=100000&offset=1&filters=ym:s:<attribution>UTMSource=='cubo'&accuracy=full&attribution=lastsign
https://beta.api-metrika.yandex.

HTTPError: HTTP Error 403: Forbidden

In [78]:
total_df.to_csv('D:\\_Docs_\\2016-06\\2016-06-06 New UTM attribution\\data.csv',encoding='utf8}')

In [3]:
domain = 'ribambelle.ru'

In [44]:
vipProjectInfo = getVipProjectByDomain(domain)
metrikaCredentials = getMetrikaCredentialsByLPD({'yandexLogin':vipProjectInfo[0]['metrikalogin'],'yandexPassword':vipProjectInfo[0]['metrikapassword'].replace('&','%26'),'domain':domain.decode("utf-8")})

In [45]:
metrikaCredentials

{'Token': u'4be1f6f321ed442bb0b930bec0a2eddf', 'counterID': [16630105]}

In [46]:
date1 = '2016-06-15'
date2 = '2016-06-16'
goalsMetrics = ','.join(['ym:s:goal'+str(target['targetid'])+'visits' for target in getCPATargetsByProjectID(vipProjectInfo[0]['projectid'])])
otherMetrics = 'ym:s:visits'
metrics = ','.join([otherMetrics,goalsMetrics])
dimensions='ym:s:specialUser,ym:s:dateTime,ym:s:<attribution>TrafficSource,ym:s:<attribution>SourceEngine,ym:s:<attribution>UTMSource,ym:s:<attribution>UTMMedium,ym:s:<attribution>UTMCampaign,ym:s:<attribution>UTMContent,ym:s:<attribution>UTMTerm'
token = metrikaCredentials['Token']
counterId = metrikaCredentials['counterID'][0]
filters = ""
attribution = 'lastsign'
df = getMetrikaReportInDataFrame(date1,date2,metrics,dimensions,token,counterId,filters,attribution)

https://beta.api-metrika.yandex.ru/stat/v1/data.csv?dimensions=ym:s:specialUser,ym:s:dateTime,ym:s:<attribution>TrafficSource,ym:s:<attribution>SourceEngine,ym:s:<attribution>UTMSource,ym:s:<attribution>UTMMedium,ym:s:<attribution>UTMCampaign,ym:s:<attribution>UTMContent,ym:s:<attribution>UTMTerm&metrics=ym:s:visits,ym:s:goal18929570visits,ym:s:goal18929580visits,ym:s:goal18929575visits,ym:s:goal16379560visits,ym:s:goal16379555visits&date1=2016-06-15&date2=2016-06-16&ids=16630105&oauth_token=4be1f6f321ed442bb0b930bec0a2eddf&limit=100000&offset=1&filters=&accuracy=full&attribution=lastsign


In [49]:
domains = ["russj.ru","organickids.ru","pinakotekaresto.com","prosto-gruz.ru","zbbb.ru","alcotek.ru","edemdom.ru","labotini.ru","realstroy.com","tencon.ru","жк-московский.com",
"народная-23.рф","doorlock.ru","stm66.ru","academoffice.ru","adel-omsk.ru","baxburo.ru","fabslog.ru","ferkon.ru","fosterland.ru","gemainvest.ru","hotel.sipk.ru",
"kerasol.ru","medstrahovka.ru","m-y.ru","nasoscentr.ru","optikdeal.ru","pickup-center.ru","proudmom.ru","ribambelle.ru","rusmaneken.ru","simple-pro.com","teploppu.ru",
"altpremium.ru","mosvipremont.ru","multifin.ru","megakran.ru","energotehmontag.ru","nanosegway.ru","mebmetall.ru","oooptr77.ru","saabparts.su","lodki-piter.ru",
"accessoriesforhome.ru","darion-svet.ru","vtb24leasing.ru","sevastopolstroy.ru"]

In [52]:
total_df = pd.DataFrame()
for domain in domains:
    vipProjectInfo = getVipProjectByDomain(domain)
    metrikaCredentials = getMetrikaCredentialsByLPD({'yandexLogin':vipProjectInfo[0]['metrikalogin'],'yandexPassword':vipProjectInfo[0]['metrikapassword'],'domain':domain.decode("utf-8")})
    date1 = '2016-05-01'
    date2 = '2016-05-31'
    goalsMetrics = ','.join(['ym:s:goal'+str(target['targetid'])+'visits' for target in getCPATargetsByProjectID(vipProjectInfo[0]['projectid'])])
    otherMetrics = 'ym:s:visits'
    metrics = ','.join([otherMetrics,goalsMetrics])
    dimensions='ym:s:deviceCategory'
    token = metrikaCredentials['Token']
    counterId = metrikaCredentials['counterID'][0]
    filters = "ym:s:<attribution>UTMMedium=='direct'"
    attribution = 'lastsign'
    df = getMetrikaReportInDataFrame(date1,date2,metrics,dimensions,token,counterId,filters,attribution)
    df['domain'] = domain
    df = dropTotalRow(mergeCPAVisitsColumns(df))
    total_df = pd.concat([total_df,df])

https://beta.api-metrika.yandex.ru/stat/v1/data.csv?dimensions=ym:s:deviceCategory&metrics=ym:s:visits,ym:s:goal20722165visits,ym:s:goal11692031visits,ym:s:goal11692036visits&date1=2016-05-01&date2=2016-05-31&ids=30722483&oauth_token=2cc70776f1ca4b06a662ec47714291e4&limit=100000&offset=1&filters=ym:s:<attribution>UTMMedium=='direct'&accuracy=full&attribution=lastsign
https://beta.api-metrika.yandex.ru/stat/v1/data.csv?dimensions=ym:s:deviceCategory&metrics=ym:s:visits,ym:s:goal19493860visits,ym:s:goal11284707visits,ym:s:goal11284702visits,ym:s:goal11086937visits&date1=2016-05-01&date2=2016-05-31&ids=29007535&oauth_token=9a21aa0f83ba43c7ba8539b91b605d35&limit=100000&offset=1&filters=ym:s:<attribution>UTMMedium=='direct'&accuracy=full&attribution=lastsign
https://beta.api-metrika.yandex.ru/stat/v1/data.csv?dimensions=ym:s:deviceCategory&metrics=ym:s:visits,ym:s:goal16248195visits&date1=2016-05-01&date2=2016-05-31&ids=34188145&oauth_token=b69353d2e4a44e0aa2960c34c839b5a3&limit=100000&offs

HTTPError: HTTP Error 403: Forbidden

In [54]:
total_df.to_csv('D:\\_Docs_\\2016-06\\device_types.csv',encoding='utf8')